## Tercer caso práctico de Modelos de Simulación
#### - Rivera Juan Matías

In [1]:
import datetime

##### Función transforma un datetime en un string con valor igual al tiempo(time)

In [2]:
def transformarTiempo(tiempo):
    st = datetime.datetime.strptime(str(tiempo), "%Y-%m-%d %H:%M:%S")
    return st.time()

#### Clase de cola

In [3]:
class Cola:
    def __init__(self,q,TiempoLlegada,DeltaLlegada,TiempoAbandono,DeltaAbandono,ListaAbandono,Limite):
        self.q=q
        self.descarte = 0
        self.tiempoL=TiempoLlegada
        self.deltaL = DeltaLlegada
        self.deltaA = DeltaAbandono
        self.contador = 0
        self.listaAbandono = ListaAbandono
        self.proxAbandono = TiempoAbandono
        self.limite = Limite
        
    def Llegada(self,PS):
        tiempoA = self.tiempoL
        if PS.estado == 0:
            PS.estado = 1
            PS.tiempoFS = self.tiempoL + PS.deltaFS[PS.contador]
            PS.contador+=1
        else:
            self.q += 1
            self.listaAbandono.append(tiempoA + self.deltaA)
            self.proxAbandono = self.listaAbandono[0]


        self.tiempoL += self.deltaL[self.contador]
        self.contador +=1
        return tiempoA
    
    def ReinicioContador(self):
        if self.contador > len(self.deltaL)-1:
            self.contador = 0
            
    def AbandonoDeCola(self):
        tiempoA = self.proxAbandono
        self.q-=1
        del self.listaAbandono[0]
        self.descarte += 1
        if self.listaAbandono:
            self.proxAbandono = self.listaAbandono[0]
        else:
            self.proxAbandono = self.limite
        return tiempoA

#### Clase de Puesto de Servicio

In [4]:
class PuestoDeServicio:
    def __init__(self,Estado,TiempoFS,DeltaFS,Limite):
        self.estado = Estado
        self.tiempoFS = TiempoFS
        self.deltaFS = DeltaFS
        self.contador = 0
        self.limite = Limite

        
    def FinDeServicio(self,Cola):
        tiempoA = self.tiempoFS
        
        if Cola.q > 0:
            Cola.q -= 1
            self.tiempoFS += self.deltaFS[self.contador]
            self.contador += 1
            del Cola.listaAbandono[0]
        else:
            self.estado = 0
            self.tiempoFS = self.limite
        if Cola.listaAbandono:
            Cola.proxAbandono = Cola.listaAbandono[0]
        else:
            Cola.proxAbandono = self.limite
            
        return tiempoA
        
    def ReinicioContador(self):
        if self.contador > len(self.deltaFS)-1:
            self.contador = 0
            


##### Función para imprimir Tiempo, recibe variables en el siguiente orden: Hora Actual, LLegada del Cliente, Fin de Servicio, N° Cola (q) y Estado del Puesto de Servicio(PS)'

In [5]:
def imprimirTiempo(Cola,PS,TiempoA):
    
    tiempoActual = transformarTiempo(TiempoA)
    tiempoLlegada = transformarTiempo(Cola.tiempoL)
    tiempoFinServicio = transformarTiempo(PS.tiempoFS)
    tiempoAbandono = transformarTiempo(Cola.proxAbandono)
    if PS.estado==0:
        tiempoFinServicio = "        "
    if Cola.q == 0:
        tiempoAbandono = "        "

    print(tiempoActual," | ",tiempoLlegada," | ",tiempoFinServicio," | ",tiempoAbandono," | ",Cola.q," | ",PS.estado," | ",Cola.descarte)

##### Función vector de deltas de tiempo por segundo'

In [6]:
def delta(tiemposDelta):
    listaDelta = []
    for i in tiemposDelta:
        listaDelta.append(datetime.timedelta(seconds = i))
    return listaDelta

##### Función asignación de tiempo'

In [7]:
def tiempo(horas,minutos,segundos):
    tiempo = datetime.datetime.combine(datetime.datetime.today(),datetime.time(horas,minutos,segundos,0)) 
    return tiempo

In [8]:
def timedelta(i):
    return datetime.timedelta(seconds = i)

##### Declaración de los tiempos iniciales y los deltas

In [9]:
#Instancia del limite(HORA DE SERVICIO)'
limite = tiempo(12,0,0)

#Instancia de los tiempos iniciales
tiempoActual = tiempo(9,0,32)
tiempoL = tiempo(9,0,49)
tiempoFS = tiempo(9,1,7)





#Tiempos delta particulares del práctico
tiempoDL = [10,5,7,7,107,24]
tiempoDFS = [50,76]
listaAb = []
tiempoAb = limite + timedelta(30)

deltaL = delta(tiempoDL)
deltaFS = delta(tiempoDFS)
deltaAb = timedelta(120)



#Instancia de la Cola y del Puesto de Servicio
cola = Cola(0,tiempoL,deltaL,tiempoAb,deltaAb,listaAb,limite+timedelta(30))
PS = PuestoDeServicio(1,tiempoFS,deltaFS,limite+timedelta(30))


##### Programa principal, un while que imprime toda la matriz.

In [10]:
print("  Actual  |  LLegada   | Fin de S.  |  Prox. Ab. |  q  |  PS | Abandonos")
print("------------------------------------------------------------------------")

#Se imprime primero el estado inicial
imprimirTiempo(cola,PS,tiempoActual)


while tiempoActual < limite:
        #Si el tiempo de llegada de cliente es menor al fin de servicio'
        if cola.tiempoL <= PS.tiempoFS  and cola.tiempoL <= cola.proxAbandono and cola.tiempoL < limite:
            
            tiempoActual = cola.Llegada(PS)  
            imprimirTiempo(cola,PS,tiempoActual)
            
        #Si el tiempo de fin de servicio es menor que el de llegada'
        elif PS.tiempoFS <= cola.tiempoL and PS.tiempoFS <= cola.proxAbandono and PS.tiempoFS < limite and PS.estado == 1:
            tiempoActual = PS.FinDeServicio(cola)
        
            imprimirTiempo(cola,PS,tiempoActual)
         
        elif cola.proxAbandono <= cola.tiempoL and cola.proxAbandono <= PS.tiempoFS and cola.proxAbandono < limite:
            tiempoActual = cola.AbandonoDeCola()
            imprimirTiempo(cola,PS,tiempoActual)
        
        
        #Cierre
        else:
            if PS.estado == 1:
                print("El último cliente en el PS saldrá a las horas:",transformarTiempo(PS.tiempoFS))
            break
        
        #Reinicio de contadores
        cola.ReinicioContador()
        PS.ReinicioContador()

print("Clientes sin atención:",cola.q)
print("Abandonos: %d" % (cola.descarte))
    

  Actual  |  LLegada   | Fin de S.  |  Prox. Ab. |  q  |  PS | Abandonos
------------------------------------------------------------------------
09:00:32  |  09:00:49  |  09:01:07  |            |  0  |  1  |  0
09:00:49  |  09:00:59  |  09:01:07  |  09:02:49  |  1  |  1  |  0
09:00:59  |  09:01:04  |  09:01:07  |  09:02:49  |  2  |  1  |  0
09:01:04  |  09:01:11  |  09:01:07  |  09:02:49  |  3  |  1  |  0
09:01:07  |  09:01:11  |  09:01:57  |  09:02:59  |  2  |  1  |  0
09:01:11  |  09:01:18  |  09:01:57  |  09:02:59  |  3  |  1  |  0
09:01:18  |  09:03:05  |  09:01:57  |  09:02:59  |  4  |  1  |  0
09:01:57  |  09:03:05  |  09:03:13  |  09:03:04  |  3  |  1  |  0
09:03:04  |  09:03:05  |  09:03:13  |  09:03:11  |  2  |  1  |  1
09:03:05  |  09:03:29  |  09:03:13  |  09:03:11  |  3  |  1  |  1
09:03:11  |  09:03:29  |  09:03:13  |  09:03:18  |  2  |  1  |  2
09:03:13  |  09:03:29  |  09:04:03  |  09:05:05  |  1  |  1  |  2
09:03:29  |  09:03:39  |  09:04:03  |  09:05:05  |  2  |  1  |

09:42:59  |  09:43:05  |  09:43:07  |  09:43:04  |  3  |  1  |  52
09:43:04  |  09:43:05  |  09:43:07  |  09:43:11  |  2  |  1  |  53
09:43:05  |  09:43:29  |  09:43:07  |  09:43:11  |  3  |  1  |  53
09:43:07  |  09:43:29  |  09:43:57  |  09:43:18  |  2  |  1  |  53
09:43:18  |  09:43:29  |  09:43:57  |  09:45:05  |  1  |  1  |  54
09:43:29  |  09:43:39  |  09:43:57  |  09:45:05  |  2  |  1  |  54
09:43:39  |  09:43:44  |  09:43:57  |  09:45:05  |  3  |  1  |  54
09:43:44  |  09:43:51  |  09:43:57  |  09:45:05  |  4  |  1  |  54
09:43:51  |  09:43:58  |  09:43:57  |  09:45:05  |  5  |  1  |  54
09:43:57  |  09:43:58  |  09:45:13  |  09:45:29  |  4  |  1  |  54
09:43:58  |  09:45:45  |  09:45:13  |  09:45:29  |  5  |  1  |  54
09:45:13  |  09:45:45  |  09:46:03  |  09:45:39  |  4  |  1  |  54
09:45:39  |  09:45:45  |  09:46:03  |  09:45:44  |  3  |  1  |  55
09:45:44  |  09:45:45  |  09:46:03  |  09:45:51  |  2  |  1  |  56
09:45:45  |  09:46:09  |  09:46:03  |  09:45:51  |  3  |  1  |

10:15:51  |  10:15:58  |  10:16:43  |  10:17:29  |  4  |  1  |  96
10:15:58  |  10:17:45  |  10:16:43  |  10:17:29  |  5  |  1  |  96
10:16:43  |  10:17:45  |  10:17:33  |  10:17:39  |  4  |  1  |  96
10:17:33  |  10:17:45  |  10:18:49  |  10:17:44  |  3  |  1  |  96
10:17:44  |  10:17:45  |  10:18:49  |  10:17:51  |  2  |  1  |  97
10:17:45  |  10:18:09  |  10:18:49  |  10:17:51  |  3  |  1  |  97
10:17:51  |  10:18:09  |  10:18:49  |  10:17:58  |  2  |  1  |  98
10:17:58  |  10:18:09  |  10:18:49  |  10:19:45  |  1  |  1  |  99
10:18:09  |  10:18:19  |  10:18:49  |  10:19:45  |  2  |  1  |  99
10:18:19  |  10:18:24  |  10:18:49  |  10:19:45  |  3  |  1  |  99
10:18:24  |  10:18:31  |  10:18:49  |  10:19:45  |  4  |  1  |  99
10:18:31  |  10:18:38  |  10:18:49  |  10:19:45  |  5  |  1  |  99
10:18:38  |  10:20:25  |  10:18:49  |  10:19:45  |  6  |  1  |  99
10:18:49  |  10:20:25  |  10:19:39  |  10:20:09  |  5  |  1  |  99
10:19:39  |  10:20:25  |  10:20:55  |  10:20:19  |  4  |  1  |

10:50:19  |  10:50:24  |  10:51:09  |  10:52:09  |  2  |  1  |  141
10:50:24  |  10:50:31  |  10:51:09  |  10:52:09  |  3  |  1  |  141
10:50:31  |  10:50:38  |  10:51:09  |  10:52:09  |  4  |  1  |  141
10:50:38  |  10:52:25  |  10:51:09  |  10:52:09  |  5  |  1  |  141
10:51:09  |  10:52:25  |  10:52:25  |  10:52:19  |  4  |  1  |  141
10:52:19  |  10:52:25  |  10:52:25  |  10:52:24  |  3  |  1  |  142
10:52:24  |  10:52:25  |  10:52:25  |  10:52:31  |  2  |  1  |  143
10:52:25  |  10:52:49  |  10:52:25  |  10:52:31  |  3  |  1  |  143
10:52:25  |  10:52:49  |  10:53:15  |  10:52:38  |  2  |  1  |  143
10:52:38  |  10:52:49  |  10:53:15  |  10:54:25  |  1  |  1  |  144
10:52:49  |  10:52:59  |  10:53:15  |  10:54:25  |  2  |  1  |  144
10:52:59  |  10:53:04  |  10:53:15  |  10:54:25  |  3  |  1  |  144
10:53:04  |  10:53:11  |  10:53:15  |  10:54:25  |  4  |  1  |  144
10:53:11  |  10:53:18  |  10:53:15  |  10:54:25  |  5  |  1  |  144
10:53:15  |  10:53:18  |  10:54:31  |  10:54:49 

11:24:45  |  11:24:49  |  11:26:01  |            |  0  |  1  |  186
11:24:49  |  11:24:59  |  11:26:01  |  11:26:49  |  1  |  1  |  186
11:24:59  |  11:25:04  |  11:26:01  |  11:26:49  |  2  |  1  |  186
11:25:04  |  11:25:11  |  11:26:01  |  11:26:49  |  3  |  1  |  186
11:25:11  |  11:25:18  |  11:26:01  |  11:26:49  |  4  |  1  |  186
11:25:18  |  11:27:05  |  11:26:01  |  11:26:49  |  5  |  1  |  186
11:26:01  |  11:27:05  |  11:26:51  |  11:26:59  |  4  |  1  |  186
11:26:51  |  11:27:05  |  11:28:07  |  11:27:04  |  3  |  1  |  186
11:27:04  |  11:27:05  |  11:28:07  |  11:27:11  |  2  |  1  |  187
11:27:05  |  11:27:29  |  11:28:07  |  11:27:11  |  3  |  1  |  187
11:27:11  |  11:27:29  |  11:28:07  |  11:27:18  |  2  |  1  |  188
11:27:18  |  11:27:29  |  11:28:07  |  11:29:05  |  1  |  1  |  189
11:27:29  |  11:27:39  |  11:28:07  |  11:29:05  |  2  |  1  |  189
11:27:39  |  11:27:44  |  11:28:07  |  11:29:05  |  3  |  1  |  189
11:27:44  |  11:27:51  |  11:28:07  |  11:29:05 

11:59:11  |  11:59:29  |  11:59:37  |  11:59:18  |  2  |  1  |  230
11:59:18  |  11:59:29  |  11:59:37  |  12:01:05  |  1  |  1  |  231
11:59:29  |  11:59:39  |  11:59:37  |  12:01:05  |  2  |  1  |  231
11:59:37  |  11:59:39  |  12:00:27  |  12:01:29  |  1  |  1  |  231
11:59:39  |  11:59:44  |  12:00:27  |  12:01:29  |  2  |  1  |  231
11:59:44  |  11:59:51  |  12:00:27  |  12:01:29  |  3  |  1  |  231
11:59:51  |  11:59:58  |  12:00:27  |  12:01:29  |  4  |  1  |  231
11:59:58  |  12:01:45  |  12:00:27  |  12:01:29  |  5  |  1  |  231
El último cliente en el PS saldrá a las horas: 12:00:27
Clientes sin atención: 5
Abandonos: 231
